In [ ]:
import os
from google.colab import userdata

# Kaggle API credentials
os.environ["KAGGLE_USERNAME"] = "bhisan"
os.environ["KAGGLE_KEY"] = userdata.get('bhisan')

In [ ]:
from kagglehub import login

login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
%pip install -q -U keras-nlp keras>=3

In [ ]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [ ]:
import keras
import keras_nlp

In [ ]:
import json

with open('constitution_nepal.json', 'r') as file:
    constitution_data = json.load(file)

documents = []
for part in constitution_data:
  if part.get('articles',0):
    head = part['title']
    for article in part['articles']:
        doc_title = head + " " + article['title']
        doc_content = article['content']
        documents.append({
            'title': doc_title,
            'content': doc_content
        })
  else:
    doc_title = part['title']
    doc_content = part['content']
    documents.append({
        'title': doc_title,
        'content': doc_content
    })


In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_1.1_instruct_2b_en")
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
data = []
template = "Title:\n{title}\n\nContent:\n{content}"
for doc in documents:

    data.append(template.format(**doc))


In [ ]:
data[:3]

["Title:\nPreamble\n\nContent:\nWE, THE SOVEREIGN PEOPLE OF NEPAL;   INTERNALIZING the people's sovereign right and right to  autonomy and self-rule, while maintaining freedom,  sovereignty, territorial integrity, national unity, independence  and dignity of Nepal;  RECALLING the g",
 'Title:\nPart-1 Article 1\n\nContent:\nConstitution as Fundamental Law: (1) This Constitution is the fundamental law of Nepal. Any law inconsistent with this Constitution shall, to the extent of such inconsistency, be void. (2) It shall be the duty of every person to uphold this Constitution.',
 'Title:\nPart-1 Article 2\n\nContent:\nSovereignty and State Power: The sovereignty and State power of Nepal shall be vested in the Nepali people. It shall be exercised in accordance with the provisions set forth in this Constitution.']

In [ ]:
prompt = template.format(
    title="Explain Part-2 of Constitution of Nepal.",
    content="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Title:
Explain Part-2 of Constitution of Nepal.

Content:
The Constitution of Nepal outlines the fundamental principles, structure, and powers of the government, including the distribution of authority and responsibilities among different organs of the State. Part-2 of the Constitution elaborates on the organization, functioning, and composition of the National Council, which plays a crucial role in representing and formulating policies of the country.

**Questions:**

a) Discuss the main functions and powers of the National Council.
b) Explain the composition of the National Council.

**Answers:**

**a) Main Functions and Powers of the National Council:**

- The National Council is the legislative body of Nepal responsible for making policies, overseeing the implementation of laws, and representing the interests of the nation in international forums.
- It acts as the supreme authority in making laws and policies on all matters within its competence.
- The National Council has the auth

In [ ]:
gemma_lm.backbone.enable_lora(rank=8)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,508,900,352 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,508,900,352 (9.35 GB)

 Trainable params: 2,727,936 (10.41 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [ ]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 512
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=5, batch_size=1)

Epoch 1/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 443s 1s/step - loss: 0.9774 - sparse_categorical_accuracy: 0.4974
Epoch 2/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 411s 1s/step - loss: 0.6326 - sparse_categorical_accuracy: 0.5792
Epoch 3/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 443s 1s/step - loss: 0.5954 - sparse_categorical_accuracy: 0.5987
Epoch 4/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 443s 1s/step - loss: 0.5637 - sparse_categorical_accuracy: 0.6156
Epoch 5/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 399s 1s/step - loss: 0.5313 - sparse_categorical_accuracy: 0.6351


In [ ]:
prompt = template.format(
    title="Explain Part-2 of Constitution of Nepal.",
    content="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Title:
Explain Part-2 of Constitution of Nepal.

Content:
(1) The Parliament shall be composed of the President and the Province- Parliament and the Provincial Assembly and shall be elected by the people on the basis and with the representation of constituencies in accordance with the federal system of government and the federal law. (2050) (2) The President shall be the Chief Executive of Nepal and shall preside over the National Assembly, the Provincial Assembly, the Federal Court, the Supreme Court, the Constitutional Bench, and shall exercise the functions of the President as provided for in the federal law and the constitution of Nepal and shall be the head of the State and shall be the Commander in Chief of Nepal. (3) The Province- Parliament shall, in the Federal Parliament of Nepal, be constituted in accordance with the federal law and the constitution of Nepal and shall consist of the Provincial Assembly and the Provincial Council.


In [ ]:
# gemma_lm.save('my_fine_tuned_model.h5')
import keras
keras.saving.save_model(gemma_lm, 'my_model.keras')